In [1]:
import os
import pandas as pd
import numpy as np
from config import Config

In [2]:
def fileDataframe(fichier, parametre):
    data = pd.read_excel(fichier)
    data_trans =data.melt(id_vars=['Station','Annee','Mois'], value_vars= data.columns[3:],var_name='Jour',value_name=parametre)
    return data_trans

In [3]:
#Creer une time serie en fusionnant l'Annee, le mois et le Jour pour creer un index de date
def convertTimeSerie(df):
    # Fonction de validation pour vérifier si une date est valide
    def dateValid(year, month, day):
        try:
            pd.Timestamp(year=year, month=month, day=day)
            return True
        except ValueError:
            return False 
    # Créer une nouvelle colonne "Date" en fusionnant les colonnes "Jour", "Mois" et "Annee"
    df['Date'] = df.apply(lambda row: str(row['Jour']) + '/' + str(row['Mois']) + '/' + str(row['Annee']), axis=1)
    # Convertir la colonne "Date" en objet datetime en appliquant la validation
    df.loc[:, 'Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y', errors='coerce')
    # Supprimer les lignes avec des dates non valides
    df = df.dropna(subset=['Date'])
     # Supprimer les colonnes "Annee", "Mois" et "Jour" si nécessaire
    df.drop(['Annee', 'Mois', 'Jour'], axis=1, inplace=True)
    return df

In [4]:
# Lecture du fichier de données journalières de pluie
pluieData=fileDataframe(str(Config.PLUIE_ETP_PATH)+'/'+str(Config.PLUIE_FILE_NAME),'Pluie')
pluieData.head(3)

,Station,Annee,Mois,Jour,Pluie
0,Bobo_Dioulasso,1907,3,01,NaN
1,Bobo_Dioulasso,1907,4,01,0.0
2,Bobo_Dioulasso,1907,5,01,2.9


In [5]:
# Lecture du fichier de données journalières d'evapotranspiration
petData=fileDataframe(str(Config.PLUIE_ETP_PATH)+'/'+str(Config.ETP_FILE_NAME),'Etp')
petData.head(3)

,Station,Annee,Mois,Jour,Etp
0,Bobo_Dioulasso,1961,1,01,6.3
1,Bobo_Dioulasso,1961,2,01,8.4
2,Bobo_Dioulasso,1961,3,01,7.7


In [6]:
petSerie=convertTimeSerie(petData)
petSerie.head(3)

/var/folders/pk/rg_m87h56ws5wzxy5dg2rcmw0000gn/T/ipykernel_1830/458809572.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['Annee', 'Mois', 'Jour'], axis=1, inplace=True)


,Station,Etp,Date
0,Bobo_Dioulasso,6.3,1961-01-01 00:00:00
1,Bobo_Dioulasso,8.4,1961-02-01 00:00:00
2,Bobo_Dioulasso,7.7,1961-03-01 00:00:00


In [7]:
pluieSerie=convertTimeSerie(pluieData)
pluieSerie.head()

/var/folders/pk/rg_m87h56ws5wzxy5dg2rcmw0000gn/T/ipykernel_1830/458809572.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['Annee', 'Mois', 'Jour'], axis=1, inplace=True)


,Station,Pluie,Date
0,Bobo_Dioulasso,NaN,1907-03-01 00:00:00
1,Bobo_Dioulasso,0.0,1907-04-01 00:00:00
2,Bobo_Dioulasso,2.9,1907-05-01 00:00:00
3,Bobo_Dioulasso,0.0,1907-06-01 00:00:00
4,Bobo_Dioulasso,13.8,1907-07-01 00:00:00


In [8]:
# Fusionner les dataframes en utilisant les colonnes "Date" et "Station" comme clés de fusion
merged_data = pd.merge(pluieSerie, petSerie, on=['Date', 'Station'], how='outer')
merged_data.to_csv(str(Config.FILES_TRAITED_PATH) + '/' + str(Config.PLUIE_ETP_FILE_NAME))
# Afficher le dataframe fusionné
merged_data.head(3)

,Station,Pluie,Date,Etp
0,Bobo_Dioulasso,NaN,1907-03-01 00:00:00,NaN
1,Bobo_Dioulasso,0.0,1907-04-01 00:00:00,NaN
2,Bobo_Dioulasso,2.9,1907-05-01 00:00:00,NaN


In [9]:
df_boromo=merged_data[merged_data['Station']=='Boromo']
df_boromo.to_csv(str(Config.FILES_TRAITED_PATH) + '/' + str(Config.STATION_TEST))